In [29]:
import os
import subprocess

METHOD = "ADAP_BARY_AREA"
parcel_type = "baillarger"#nieuwenhuys
# nieuwenhuys_parcellation using the MYATLAS package
myatlas_base_path = "/Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/"
std_fslr_sphere_path = "/Users/dennis.jungchildmind.org/OneDrive - Child Mind Institute/layer_project/cortical_layer/jupyter-notebook/082725_myelo_parcellate/standard_mesh_atlases"

# subpaths
parc_subpath = "maps/Surface/Freesurfer_COLIN27/gii/parcellation"
current_sphere_subpath = "cortical_surface/COLIN27_FS/"
surf_subpath = "COLIN27_FS/surf"

# Both hemispheres
hemispheres = ["lh", "rh"]
subjects_base_folder = "/Users/dennis.jungchildmind.org/Desktop/exvivo"

# Get list of subject directories (filter out hidden files and non-directories)
subjects = [d for d in os.listdir(subjects_base_folder) 
           if os.path.isdir(os.path.join(subjects_base_folder, d)) and not d.startswith('.')]

print(f"Found {len(subjects)} subjects: {subjects}")
print(f"Will check for hemispheres: {hemispheres}")
print("=" * 70)

total_processed = 0
total_skipped = 0

for i, subject in enumerate(subjects, 1):
    print(f"\nProcessing subject {i}/{len(subjects)}: {subject}")
    print("-" * 50)
    
    subject_folder = os.path.join(subjects_base_folder, subject)
    subject_processed = False
    
    # Check each hemisphere
    for hemisphere in hemispheres:
        print(f"\n  Checking hemisphere: {hemisphere}")
        print("  " + "-" * 30)
        
        # Subject-specific files for this hemisphere
        target_subject_file = os.path.join(subject_folder, f"{hemisphere}.inf.32k_fs_LR.surf.gii")
        
        # Check if the subject's hemisphere file exists
        if not os.path.exists(target_subject_file):
            print(f"  ⚠️  Skipping {hemisphere} - Subject surface file not found: {target_subject_file}")
            continue
        
        # Create subject-specific output folder
        output_folder = os.path.join(subject_folder, "niu_parcellation_output")
        
        # Atlas files for this hemisphere
        #niu_parc_file = os.path.join(myatlas_base_path, parc_subpath, f"nieuwenhuys_parcellation_on_colin27_{hemisphere}.label.gii")
        parc_file = []
        if parcel_type.lower() == "nieuwenhuys":
            parc_file = os.path.join(myatlas_base_path, parc_subpath, f"nieuwenhuys_parcellation_on_colin27_{hemisphere}.label.gii")
        elif parcel_type.lower() == "baillarger":
            parc_file = os.path.join(myatlas_base_path, parc_subpath, f"Baillarger_type_parcellation_on_colin27_{hemisphere}.label.gii")
       
        current_sphere_file = os.path.join(myatlas_base_path, surf_subpath, f"{hemisphere}.sphere.reg.reg_LR.native.surf.gii")

        if hemisphere == "lh":
            target_sphere_file = os.path.join(std_fslr_sphere_path, "L.sphere.32k_fs_LR.surf.gii")
        else:
            target_sphere_file = os.path.join(std_fslr_sphere_path, "R.sphere.32k_fs_LR.surf.gii")
        
        # area-surfs
        white_surf = os.path.join(myatlas_base_path, surf_subpath, f"{hemisphere}.white.gii")
        
        # current-roi
        roi_file = os.path.join(myatlas_base_path, surf_subpath, f"{hemisphere}.roi.shape.gii")
        
        label_output = []
        if parcel_type.lower() == "nieuwenhuys":
            label_output = f"{subject}_{hemisphere}_nieuwenhuys_parcellation.label.gii"
        elif parcel_type.lower() == "baillarger":
            label_output = f"{subject}_{hemisphere}_baillarger_parcellation.label.gii"
        
        # Create output directory if it doesn't exist
        try:
            os.makedirs(output_folder, exist_ok=True)
            output_path = os.path.join(output_folder, label_output)
        except OSError as e:
            print(f"  ✗ Error creating output directory for {subject} {hemisphere}: {e}")
            continue
        
        '''
        # Check if output already exists
        if os.path.exists(output_path):
            print(f"  ⚠️  Output already exists for {hemisphere}: {output_path}")
            print(f"  ℹ️  Skipping {hemisphere} (delete file to reprocess)")
            continue
        '''
        # Check if all input files exist before proceeding
        input_files = {
            "Parcellation file": parc_file,
            "Current sphere file": current_sphere_file,
            "Target sphere file": target_sphere_file,
            "White surface": white_surf,
            "Target subject surface": target_subject_file,
            "ROI file": roi_file
        }
        
        missing_files = []
        for file_desc, file_path in input_files.items():
            if not os.path.exists(file_path):
                missing_files.append(f"{file_desc}: {file_path}")
        
        if missing_files:
            print(f"  ✗ Skipping {hemisphere} - Missing files:")
            for missing in missing_files:
                print(f"    - {missing}")
            continue
        
        # Construct the wb_command
        wb_command = [
            "wb_command",
            "-label-resample",
            parc_file,
            current_sphere_file,
            target_sphere_file,
            METHOD,
            output_path,
            "-area-surfs",
            white_surf,
            target_subject_file,
            "-current-roi",
            roi_file
        ]
        
        # Print the command for verification
        print(f"  🔄 Running command for {subject} {hemisphere}:")
        print(f"     {' '.join(wb_command)}")
        print()
        
        try:
            # Run the command
            result = subprocess.run(wb_command, check=True, capture_output=True, text=True)
            print(f"  ✓ Command executed successfully for {subject} {hemisphere}!")
            print(f"    Output saved to: {output_path}")
            if result.stdout:
                print(f"    Output: {result.stdout}")
            if result.stderr:
                print(f"    Stderr: {result.stderr}")
            
            subject_processed = True
            total_processed += 1
            
        except subprocess.CalledProcessError as e:
            print(f"  ✗ Error executing command for {subject} {hemisphere}: {e}")
            print(f"    Return code: {e.returncode}")
            if e.stdout:
                print(f"    Stdout: {e.stdout}")
            if e.stderr:
                print(f"    Stderr: {e.stderr}")
            
        except FileNotFoundError:
            print("  ✗ Error: wb_command not found. Make sure Connectome Workbench is installed and in your PATH.")
            print("Stopping processing...")
            exit(1)
    
    if not subject_processed:
        print(f"  ⚠️  No hemispheres processed for subject {subject}")
        total_skipped += 1

print(f"\n{'='*70}")
print("PROCESSING SUMMARY")
print(f"{'='*70}")
print(f"Total subjects found: {len(subjects)}")
print(f"Total hemisphere files processed: {total_processed}")
print(f"Total subjects with no hemispheres processed: {total_skipped}")
print(f"Processing complete!")

Found 17 subjects: ['I57_new_confidence', 'I58_new_confidence', 'I45_new_confidence', 'I59_new_confidence', 'I56_new_confidence', 'I41_new_confidence', 'I54_new_confidence', 'I38_new_confidence', 'I53_new_confidence', 'I46_new_confidence', 'EXC022_new_confidence', 'I52_new_confidence', 'I48_new_confidence', 'I55_new_confidence', 'I61_new_confidence', 'KC001_new_confidence', 'I60_new_confidence']
Will check for hemispheres: ['lh', 'rh']

Processing subject 1/17: I57_new_confidence
--------------------------------------------------

  Checking hemisphere: lh
  ------------------------------
  🔄 Running command for I57_new_confidence lh:
     wb_command -label-resample /Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/maps/Surface/Freesurfer_COLIN27/gii/parcellation/Baillarger_type_parcellation_on_colin27_lh.label.gii /Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/COLIN27_FS/surf/lh.sphere.reg.reg_LR.native.surf.gii /Users/dennis.jungchildmind.org/OneDrive - Chi